In [1]:
pip install transformers datasets torch scikit-learn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00


## Dataset for Fine-Tuning

In [2]:
{
    'article': "Text of the article.",
    'highlights': "Summary of the article."
}

{'article': 'Text of the article.', 'highlights': 'Summary of the article.'}

In [3]:
from datasets import load_dataset

# Load CNN/DailyMail dataset for training
dataset = load_dataset("cnn_dailymail", "3.0.0", split='train[:1%]')
print(dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

## Set Up the Model and Tokenizer

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize tokenizer and model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Prepare Data for Fine-Tuning

In [10]:
import torch

def preprocess_data(batch):
    # Add "summarize: " prompt to the article text
    inputs = ["summarize: " + article for article in batch['article']]

    # Tokenize the input articles
    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )

    # Tokenize the target summaries
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch['highlights'], max_length=128, truncation=True, padding="max_length"
        )

    # Convert to PyTorch tensors
    model_inputs['input_ids'] = torch.tensor(model_inputs['input_ids'])
    model_inputs['attention_mask'] = torch.tensor(model_inputs['attention_mask'])
    model_inputs['labels'] = torch.tensor(labels['input_ids'])

    return model_inputs

## Set Up Data Loaders

In [11]:
from torch.utils.data import DataLoader

# Map the preprocessing function and convert to DataLoader
processed_data = dataset.map(preprocess_data, batched=True)

# Load DataLoader with processed data
train_dataloader = DataLoader(processed_data, batch_size=8, shuffle=True)

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

## Define Optimizer and Scheduler

In [12]:
from transformers import AdamW
from torch.optim.lr_scheduler import StepLR

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Scheduler to reduce learning rate over time
scheduler = StepLR(optimizer, step_size=1000, gamma=0.1)

##  Fine-Tuning the Model

In [ ]:
from torch.utils.data import DataLoader
import torch

for epoch in range(3):  # Example: 3 epochs
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        # Move tensors to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Debugging: Print tensor shapes
        print(f"Input IDs: {input_ids.shape}, Attention Mask: {attention_mask.shape}, Labels: {labels.shape}")

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Loss: {avg_loss:.4f}")

## Evaluating the Fine-Tuned Model

In [ ]:
model.eval()

# Function to generate a summary from the model
def generate_summary(text):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    inputs = inputs.to(device)

    summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Test the fine-tuned model
text = """
 Artificial intelligence lends an extremely helping hand when attempting to make
predictions with your data. We’ll talk more about this type of analysis later, but predictive
analysis and the above list aren’t an exhaustive list of the uses of AI. Artificial intelligence
can help you and your team with data analytics in hundreds of different ways. Before
you jump into implementing AI into your process, let's look at some of the benefits and
challenges of AI.
"""

print("Generated Summary:", generate_summary(text))

## Save and Load the Fine-Tuned Model

In [ ]:
# Save the fine-tuned model to disk
model.save_pretrained("fine_tuned_t5")
tokenizer.save_pretrained("fine_tuned_t5")

# Load the fine-tuned model later
model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5")
tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5")

##  Evaluate the Model with ROUGE Score

In [ ]:
pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

# Sample summaries
reference_summary = "ChatGPT is an NLP model developed by OpenAI."
generated_summary = generate_summary(text)

# Calculate ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, generated_summary)

print("ROUGE Scores:", scores)

## This project demonstrates:

### Fine-tuning a transformer model for text summarization.
### Handling tokenization and large datasets with Hugging Face.
### Evaluating performance using ROUGE scores